In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import Ridge


In [2]:
dt = pd.read_csv('../data/nam_processed_data.csv')
print(dt.head())

   bhk  area  status  price_in_USD  house_type  new  region_Agripada  \
0    2   650       1      314400.0           0    0             True   
1    1   803       0      270000.0           0    0             True   
2    1   684       0      204000.0           0    0             True   
3    1   545       0      192000.0           0    0             True   
4    2  1650       0      420000.0           0    0             True   

   region_Airoli  region_Ambernath East  region_Ambernath West  ...  \
0          False                  False                  False  ...   
1          False                  False                  False  ...   
2          False                  False                  False  ...   
3          False                  False                  False  ...   
4          False                  False                  False  ...   

   region_Vangani  region_Vasai  region_Vashi  region_Vikhroli  \
0           False         False         False            False   
1       

In [3]:
print(dt.shape)

(45343, 115)


In [4]:
print(dt.head())

   bhk  area  status  price_in_USD  house_type  new  region_Agripada  \
0    2   650       1      314400.0           0    0             True   
1    1   803       0      270000.0           0    0             True   
2    1   684       0      204000.0           0    0             True   
3    1   545       0      192000.0           0    0             True   
4    2  1650       0      420000.0           0    0             True   

   region_Airoli  region_Ambernath East  region_Ambernath West  ...  \
0          False                  False                  False  ...   
1          False                  False                  False  ...   
2          False                  False                  False  ...   
3          False                  False                  False  ...   
4          False                  False                  False  ...   

   region_Vangani  region_Vasai  region_Vashi  region_Vikhroli  \
0           False         False         False            False   
1       

In [48]:
X = dt.drop(['price_in_USD'], axis = 'columns')
Y = dt.price_in_USD


In [50]:
print(Y.shape)

(45343,)


In [52]:
from sklearn.linear_model import LassoCV, Lasso
from sklearn.datasets import make_regression
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import RepeatedKFold
idx = np.arange(X.shape[0])
X_train, X_test, Y_train, Y_test, idx_train, idx_test = train_test_split(X , Y , idx,test_size = 0.2, random_state=10)

# Khởi tạo pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', Lasso())
])

# Không gian tham số alpha
param_grid = {'model__alpha': np.arange(0, 1, 0.05)}

# Grid Search trên tập huấn luyện (X_train, Y_train)
grid_search = GridSearchCV(pipeline, param_grid, scoring='neg_mean_squared_error', cv=5)  # Sử dụng 5-fold CV trên tập huấn luyện
grid_search.fit(X_train, Y_train)

# Lấy mô hình tốt nhất và dự đoán trên tập kiểm tra
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Đánh giá mô hình trên tập kiểm tra
mse_test = mean_squared_error(Y_test, y_pred)
r2_test = r2_score(Y_test, y_pred)

print(f"Best alpha: {grid_search.best_params_['model__alpha']}")
print(f"MSE on test set: {mse_test:.4f}")
print(f"R-squared on test set: {r2_test:.4f}")

# In ra độ chính xác (R-squared) trên tập huấn luyện
r2_train = best_model.score(X_train, Y_train)
print(f'R-squared trên tập huấn luyện: {r2_train:.4f}')

# Hệ số hồi quy và hệ số chặn của mô hình tốt nhất
print("Coefficients:", best_model.named_steps["model"].coef_)
print("Intercept:", best_model.named_steps["model"].intercept_)

c:\Users\Admin\anaconda3\Lib\site-packages\sklearn\pipeline.py:405: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
c:\Users\Admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
c:\Users\Admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.273e+13, tolerance: 1.277e+11 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.

Best alpha: 0.0
MSE on test set: 5516695647.0411
R-squared on test set: 0.8676
R-squared trên tập huấn luyện: 0.8564
Coefficients: [-2.42528314e+04  1.48524203e+05  4.13088492e+03  1.85200638e+03
  4.63375001e+03  5.13026682e+03 -1.29356939e+03 -6.86375672e+03
 -5.16309888e+03  8.91969793e+03  2.04824375e+04  5.35454154e+02
 -5.63962704e+03 -4.37866910e+03  1.95746265e+04  6.77830294e+03
  3.09832934e+04 -9.23032294e+02  5.53862027e+02  3.10943794e+03
 -4.73075966e+03 -2.37642319e+03 -5.13855984e+03 -3.85038628e+03
  7.11969149e+03  8.34867665e+03  2.33236687e+04  1.34540711e+04
  5.22778557e+03  1.46705126e+04  1.48250918e+04  1.88049891e+03
  6.73333502e+03 -1.87075031e+03 -1.67708913e+04 -3.34899945e+03
 -2.88494119e+03  8.99165307e+03  2.06990233e+03  2.06960797e+03
  4.28378908e+03  7.76955438e+03  4.06642907e+03  3.71822227e+03
  2.38037114e+04  3.47405916e+03 -5.89760376e+03 -2.28463517e+02
 -2.77154363e+03 -1.06273576e+04 -7.85540327e+03  8.23686431e+03
  7.94515032e+03  8.3437

c:\Users\Admin\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.106e+14, tolerance: 1.540e+11 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


In [53]:
print(Y_test)

24961    105912.0
40922     75588.0
12323     40800.0
5755     442800.0
15411    388800.0
           ...   
9629     264000.0
7325     102000.0
1218     178800.0
31936    103200.0
3093     420000.0
Name: price_in_USD, Length: 9069, dtype: float64


In [54]:
pred_table = pd.DataFrame(data={"actual value":Y_test, "predicted":best_model.predict(X_test)})
pred_table["difference"] = pred_table["actual value"] - pred_table["predicted"]
pred_table

,actual value,predicted,difference
24961,105912.0,175117.127082,-69205.127082
40922,75588.0,45172.836386,30415.163614
12323,40800.0,9790.115901,31009.884099
5755,442800.0,570487.690991,-127687.690991
15411,388800.0,543960.396843,-155160.396843
...,...,...,...
9629,264000.0,301295.090612,-37295.090612
7325,102000.0,100541.298421,1458.701579
1218,178800.0,136885.113681,41914.886319
31936,103200.0,129156.824794,-25956.824794


In [32]:
from sklearn.linear_model import Lasso
reg_lasso = Lasso(alpha = 0.55)
reg_lasso.fit(X_train, Y_train)

# Sai số huấn luyện trên tập train
print(reg_lasso.score(X_train, Y_train))

# Hệ số hồi qui và hệ số chặn
print(reg_lasso.coef_)
print(reg_lasso.intercept_)

0.9992661187839041
[-2.16045431e-01  0.00000000e+00  9.00033528e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  3.01945614e-02  7.36479696e+00
  8.90449551e-02 -0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -2.21322439e-01  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -1.42157353e-01
  0.00000000e+00  0.00000000e+00 -1.96341013e-01  0.00000000e+00
 -0.00000000e+00  7.31380235e+01 -2.25049395e-01  0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.36838233e-01
 -0.00000000e+00  3.94279017e+01 -0.00000000e+00 -0.00000000e+00
  5.23526144e+01  9.37215130e-01  4.36112519e+01  0.00000000e+00
  0.00000000e+00  0.00000000e+00  1.55449316e-01  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
  3.21676127e+01 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00